In [2]:
import os
os.chdir('../..')

In [3]:
import pandas as pd
import numpy as np

from datetime import datetime, date
import matplotlib.pyplot as plt

import json

data = 'data/'

In [4]:
atms = pd.read_csv(data + 'atms.small.csv')
clients = pd.read_csv(data + 'clients.small.csv')
companies = pd.read_csv(data + 'companies.small.csv')
transactions = pd.read_csv(data + 'transactions.small.csv')

In [281]:
months = {}
days = {}
counts = 0
dates = []
for i in transactions.index:
    date = transactions.loc[i]['date']
    month = int(date.split('/')[1])
    day = int(date.split('/')[0])
    
    if month not in months.keys():
        months[month] = 0
    
    months[month] += 1
    if day not in days.keys():
        days[day] = 0
    
    days[day] += 1
    
    if day > 12 and month > 12:
        counts += 1

In [282]:
counts/len(transactions)

0.11905019472090005

In [54]:
months

{1: 618,
 2: 622,
 3: 663,
 4: 527,
 5: 518,
 6: 661,
 7: 564,
 8: 570,
 9: 627,
 10: 450,
 11: 505,
 12: 507,
 13: 759,
 14: 552,
 15: 575,
 16: 612,
 17: 843,
 18: 675,
 19: 579,
 20: 922,
 21: 612,
 22: 709,
 23: 752,
 24: 556,
 25: 503,
 26: 474,
 27: 646,
 28: 515,
 29: 558,
 30: 579,
 31: 235}

In [57]:
days

{1: 1246,
 2: 1301,
 3: 1188,
 4: 1265,
 5: 1283,
 6: 1368,
 7: 1241,
 8: 1317,
 9: 1307,
 10: 1270,
 11: 1284,
 12: 1542,
 13: 49,
 14: 52,
 15: 43,
 16: 49,
 17: 46,
 18: 40,
 19: 45,
 20: 45,
 21: 37,
 22: 44,
 23: 41,
 24: 233,
 25: 524,
 26: 573,
 27: 1055}

## Extract time and flow patterns

In [10]:
threshold_minutes = 15
threshold_amount = 0.1

In [11]:
groups = {'source': {'time': [], 'flow': []},
          'target': {'time': [], 'flow': []}}

ids = {'source': transactions['source'].unique(), 'target': transactions['target'].unique()}

for key in ids.keys():
    print('Check for {}'.format(key))
    for id_ in ids[key]:
        df = transactions[transactions[key] == id_]
        df.index = range(len(df))

        times = np.zeros((len(df), len(df)), dtype=np.bool)
        amounts = np.zeros((len(df), len(df)), dtype=np.bool)


        for j in range(len(df)):
            for k in range(j, len(df)):
                # Time diff
                tj = datetime.strptime(df.loc[j]['time'], '%H:%M:%S').time()
                tk = datetime.strptime(df.loc[k]['time'], '%H:%M:%S').time()

                diff = np.abs(((datetime.combine(date.today(), tj) - datetime.combine(date.today(), tk)).total_seconds()))

                if diff < threshold_minutes*60:
                    times[j][k] = True
                    times[k][j] = True
                else:
                    times[j][k] = False
                    times[k][j] = False

                # Amount diff

                amj = df.loc[j]['amount']
                amk = df.loc[k]['amount']

                diff = np.abs(amj-amk)
                avg = np.mean([amj, amk])

                if diff < threshold_amount*avg:
                    amounts[j][k] = True
                    amounts[k][j] = True
                else:
                    amounts[j][k] = False
                    amounts[k][j] = False

        mat = amounts*times
        
        j=0
        while j <= len(df)-1:
            arr = [i for i, x in enumerate(mat[j, :]) if x]
            j = arr[-1]+1
            if len(arr) > 2:
                linked = [df.loc[k]['id'] for k in arr]
                if len(df[df['id'].isin(linked)]['date'].unique()) == 1:
                    groups[key]['flow'].append(linked)
                else:
                    groups[key]['time'].append(linked) 

Check for source
Check for target


In [12]:
for key in groups.keys():
    print('{}:'.format(key))
    for type_ in groups[key].keys():
        print('  {}: {}'.format(type_, len(groups[key][type_])))

source:
  time: 9
  flow: 100
target:
  time: 16
  flow: 158


# Check for similar transactions

In [13]:
for key in groups.keys():
    for type_ in groups[key].keys():
        
        ids = []
        dup = []
        for i, g in enumerate(groups[key][type_]):
            df = transactions[transactions['id'].isin(g)]
            id_ = df[key].iloc[0]

            if id_ in ids:
                dup.append([ids.index(id_), i])
            ids.append(id_)  
            
        merge = []
        for pair in dup:
            amount1 = np.mean(transactions[transactions['id'].isin(groups[key][type_][pair[0]])]['amount'])
            amount2 = np.mean(transactions[transactions['id'].isin(groups[key][type_][pair[1]])]['amount'])

            delta = np.abs(amount1-amount2)

            if delta < threshold_amount*np.mean([amount1, amount2]):
                merge.append(pair)
                
        to_del = []
        for pair in merge:
            groups[key][type_][pair[0]] = list(np.append(groups[key][type_][pair[0]], groups[key][type_][pair[1]]))

            to_del.append(pair[1])
            
        for index in sorted(to_del, reverse=True):
            del groups[key][type_][index]

In [14]:
for key in groups.keys():
    print('{}:'.format(key))
    for type_ in groups[key].keys():
        print('  {}: {}'.format(type_, len(groups[key][type_])))

source:
  time: 9
  flow: 79
target:
  time: 14
  flow: 131


In [15]:
with open(data + 'groups.json', 'w') as outfile:
    json.dump(groups, outfile)

# Extract JSON for d3js

In [5]:
with open(data + 'groups.json', 'r') as outfile:
    groups = json.load(outfile)
    
source_ids = []
for g in groups['source']['time']:
    susp_node = transactions[transactions['id'].isin(g)]['source'].iloc[0]
    source_ids.append(susp_node)
    
target_ids = []
for g in groups['target']['time']:
    susp_node = transactions[transactions['id'].isin(g)]['source'].iloc[0]
    target_ids.append(susp_node)

In [6]:
transactions[transactions['id'].isin(groups['source']['flow'][0])]

,id,source,target,date,time,amount,currency
1,59328a0d-92bb-43a5-8925-5db6a693dde9,c81d20dd-8b65-43d1-bb07-6988fffa299f,d84e8a14-d531-497c-87cb-ee985099e9f9,12/26/2016,03:57:40,940.451548,IRR
1341,f5a69e48-09c2-49a7-a911-4529bada023e,c81d20dd-8b65-43d1-bb07-6988fffa299f,1f82df7d-5d1c-457f-9c24-b0b56da86bc0,12/26/2016,03:57:37,940.451548,None
1585,46b7278c-ccc0-464c-9e2c-cca89e6b82dc,c81d20dd-8b65-43d1-bb07-6988fffa299f,6e91725e-699f-4957-9d44-a17d733ba070,12/26/2016,03:57:43,940.451548,None
2426,d28fa994-4ca5-4305-a34d-0f4c2dda7b36,c81d20dd-8b65-43d1-bb07-6988fffa299f,2132a902-7930-405a-aca0-da0160c06b4e,12/26/2016,11:57:59,940.451548,None
2451,2da4f0fc-2f0d-4b5a-a8e6-86cc74727769,c81d20dd-8b65-43d1-bb07-6988fffa299f,ec4cfb9a-641d-4af3-a18f-3c26bed31ad8,12/26/2016,11:57:02,940.451548,None
2630,72869df8-4a1b-42d6-b30d-73dcfdb215c7,c81d20dd-8b65-43d1-bb07-6988fffa299f,2b6d52d9-7a20-483c-950b-176f2d79c7be,12/26/2016,11:57:56,940.451548,None


In [7]:
transactions[transactions['id'].isin(groups['target']['flow'][0])]

,id,source,target,date,time,amount,currency
0,0a6dfc9d-c66e-4546-a2d7-3e910e6478e3,a9c9e8db-c796-4d87-bdba-bbc9bf683d40,98fdf8fb-ec37-42ed-a890-d862ef93e7bb,05/24/2017,00:47:36,457.952172,ISK
110,6a63e069-35dd-42d4-ad7b-c032c4906f2c,999aeafd-bb36-47d5-9875-bb8271d002b2,98fdf8fb-ec37-42ed-a890-d862ef93e7bb,05/24/2017,00:47:24,457.952172,XUA
183,dbc7647e-a122-431b-a5f7-69cded41045e,feb37038-d91c-4747-bb01-b7defe0c8a67,98fdf8fb-ec37-42ed-a890-d862ef93e7bb,05/24/2017,00:47:48,457.952172,None
1075,1fa01a8c-8936-4819-ab1a-ec995cdb2ab7,ad964847-eecb-4e52-95fc-a3d05a6bc3f7,98fdf8fb-ec37-42ed-a890-d862ef93e7bb,05/24/2017,00:47:12,457.952172,None
2054,940d8c41-2e5c-48c4-9e49-dd86177b8b67,a3b9bdf1-da56-4564-92be-728cef460ccf,98fdf8fb-ec37-42ed-a890-d862ef93e7bb,05/24/2017,00:47:24,457.952172,BYR
2836,29f30884-8b21-436d-80f2-90f8c5be3063,743189b4-6ad1-4c66-97ac-4fb4072cc1d0,98fdf8fb-ec37-42ed-a890-d862ef93e7bb,05/24/2017,00:47:00,457.952172,None


In [5]:
# group 0 = suspect
# group 1 = accomplice
# group 2 = accomplice edge
# group 3 = normal
# group 4 = normal edge

In [7]:
for id_ in range(3):
    data = {'nodes': [], 'links': [], 'type': None}
    
    susp_node = transactions[transactions['id'].isin(groups['source']['time'][id_])]['source'].iloc[0]
    data['nodes'].append({'id': susp_node, 'tag': 'suspect', 'type': 'suspect'})
    
    susp_index = list(transactions[transactions['id'].isin(groups['source']['time'][id_])].index)

    # get the out nodes
    out = transactions[transactions['source'] == susp_node]

    out_nodes = []

    for i in out.index:
        row = out.loc[i]
        acc = True if i in susp_index else False
        if acc:
            tag = 'accomplice'
        else:
            tag = 'usual'

        if row['target'] not in out_nodes:
            data['nodes'].append({'id': row['target'], 'tag': tag, 'type': 'target'})
            out_nodes.append(row['target'])

        data['links'].append({'source': susp_node, 'target': row['target'], 'tag': tag, 'date': row['date'], 'time': row['time'], 'amount': row['amount'], 'currency': row['currency']})
        
    in_ = transactions[transactions['target'] == susp_node]

    in_nodes = []
    if susp_node in target_ids:
        susp_in = True

        data['type'] = 'Time Pattern Outflow + Inflow'

        idx = target_ids.index(susp_node)

        susp_index_in = list(transactions[transactions['id'].isin(groups['target']['time'][idx])].index)
    else:
        data['type'] = 'Time Pattern Outflow'
        susp_in = False

    for i in in_.index:
        row = in_.loc[i]

        if susp_in:
            acc = True if i in susp_index_in else False
            if acc:
                tag = 'accomplice'
            else:
                tag = 'usual'
        else:
            tag = 'usual'

        if row['source'] not in in_nodes:
            data['nodes'].append({'id': row['source'], 'tag': tag, 'type': 'source'})  
            in_nodes.append(row['source'])

        data['links'].append({'source': row['source'], 'target': susp_node, 'tag': tag, 'date': row['date'], 'time': row['time'], 'amount': row['amount'], 'currency': row['currency']})
        
    # Save the file
    fn = str(susp_node) + '.json'

    with open('json/'+fn, 'w') as outfile:
        json.dump(data, outfile)

In [35]:
list_ = []

for id_ in range(3):
    data = {'nodes': [], 'links': [], 'type': None}
    
    susp_node = transactions[transactions['id'].isin(groups['source']['time'][id_])]['source'].iloc[0]
    
    score = np.random.random()
    
    list_.append({'id': susp_node, 'score': score})
    
list_ = sorted(list_, key=lambda k: k['score'], reverse=True)

with open('json/ranked_list.json', 'w') as outfile:
    json.dump(list_, outfile)  

In [36]:
list_

[{'id': 'd6af4bd8-f5e2-4419-b0dc-4ec1e31d611a', 'score': 0.6116555067543284},
 {'id': '7da7fe88-98be-4553-b006-6df8bbdb6945', 'score': 0.5559632804814437},
 {'id': 'c019c971-bdf3-4fe9-9f71-5d5871d8794c', 'score': 0.42804024485180914}]